In [1]:
# dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import create_engine, func
from sqlalchemy import create_engine, inspect

In [2]:
phytoplankton = pd.read_csv("data/phytoplankton.csv")
zooplankton =  pd.read_csv("data/zooplankton.csv")
phytoplankton_color = pd.read_csv("data/phytoplankton_ci.csv")

In [3]:

phytoplankton.head()

,FID,id,Route,Latitude,Longitude,SampleDateUTC,Year,Month,Day,Time_24hr,...,caab_code,taxon_start_date,taxon_per_m3,phyto_comments,acknowledgements,first_occurrence,parent_taxon_name,training,comments,geom
0,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880720,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,50566000.0,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
1,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880722,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030054.0,NaN,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (135.33099 -37.91378)
2,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880721,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,2,27,02:01:35,...,52030900.0,2011-05-17,666.7,*Noted as present in the zoop count*,Wallenius Wilhelmsen,2011-09-20,Tripos spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (135.33099 -37.91378)
3,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880726,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,99530018.0,2011-05-17,6666.7,NaN,Wallenius Wilhelmsen,2011-09-15,Pseudo-nitzschia spp.,"Internal training Pru Bonhma, Gustaaf Hallegraeff",NaN,POINT (134.91247 -37.87656)
4,soop_auscpr_phyto_flat_map.fid--404dafa0_17a9a...,30880727,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,2,27,03:10:21,...,NaN,NaN,3333.3,NaN,Wallenius Wilhelmsen,NaN,NaN,NaN,NaN,POINT (134.91247 -37.87656)


In [4]:
# get list of unique routes
column_values = phytoplankton[["Route"]].values.ravel()
unique_routes =  pd.unique(column_values)

print(unique_routes)

['ABAB' 'ABFR' 'ADAB' 'ADME' 'ANHO' 'AUHO' 'BRER' 'BRFI' 'BRGL' 'BRNC'
 'BRPK' 'BRSY' 'BUNE' 'CAGL' 'EDMI' 'FRAN' 'FRBO' 'FRSY' 'GLTO' 'HOAN'
 'HOFR' 'HOSO' 'HOSY' 'IIOE' 'MEAB' 'MEAD' 'MEFR' 'MEPK' 'NCME' 'NEBU'
 'PKME' 'SOHO' 'SYBR' 'SYHO' 'SYME' 'TINT' 'TOCA']


In [5]:
#total count of unique values

item_counts = phytoplankton["Route"].value_counts()
print(item_counts)

HOAN    15806
ANHO     6284
BRSY     3470
BRPK     2647
SYME     2098
PKME     1952
MEAD     1777
CAGL     1084
ADAB      969
ABFR      743
HOFR      735
MEAB      630
FRBO      554
GLTO      432
HOSY      406
FRAN      390
BRNC      364
BRGL      335
IIOE      315
BRER      299
SYBR      199
BUNE      191
MEFR      190
NCME      182
FRSY      172
SYHO      160
AUHO      147
TINT      144
HOSO      114
EDMI      105
NEBU       98
ABAB       90
MEPK       83
SOHO       55
BRFI       55
TOCA       49
ADME       33
Name: Route, dtype: int64


In [6]:
#for loop to check which routes to drop

for route in unique_routes:
    print(route)
    route_filter = phytoplankton.loc[phytoplankton['Route'] == route]
    print(route_filter["Year"].value_counts())
    print("***************************************************")

ABAB
2020    90
Name: Year, dtype: int64
***************************************************
ABFR
2014    300
2016    220
2017    136
2015     87
Name: Year, dtype: int64
***************************************************
ADAB
2014    488
2016    330
2017     92
2015     59
Name: Year, dtype: int64
***************************************************
ADME
2019    33
Name: Year, dtype: int64
***************************************************
ANHO
2010    1432
2009    1327
2008    1148
2018    1132
2011     702
2013     336
2017     103
2016      79
2019      24
2007       1
Name: Year, dtype: int64
***************************************************
AUHO
2011    147
Name: Year, dtype: int64
***************************************************
BRER
2014    299
Name: Year, dtype: int64
***************************************************
BRFI
2012    55
Name: Year, dtype: int64
***************************************************
BRGL
2014    281
2015     54
Name: Year, dtype: int64
*******

In [7]:
#drop unneeded columns for phytoplankton
phytoplankton = phytoplankton.drop(["id",
                                    "Time_24hr",
                                    "FID",
                                    "Month",
                                    "Day",
                                    "family",
                                    "genus",
                                    "species",
                                    "caab_code",
                                   "taxon_start_date",
                                    "taxon_name",
                                   "phyto_comments",
                                   "acknowledgements",
                                   "first_occurrence",
                                   "parent_taxon_name",
                                   "training",
                                   "comments",
                                    "geom"
                                  ],
                                  axis=1)

In [8]:
phytoplankton.head()

,Route,Latitude,Longitude,SampleDateUTC,Year,taxon_group,taxon_per_m3
0,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,Ciliate,3333.3
1,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,Dinoflagellate,666.7
2,ABAB,-37.9138,135.3310,2020-02-27T02:01:35Z,2020,Dinoflagellate,666.7
3,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,Pennate diatom,6666.7
4,ABAB,-37.8766,134.9125,2020-02-27T03:10:21Z,2020,Dinoflagellate,3333.3


In [ ]:
phyto_main_routes = phytoplankton.loc[((phytoplankton['Route'] == "ANHO") |
        (phytoplankton["Route"] == 'BRSY')|
        (phytoplankton["Route"] == 'HOAN')|
#        (phytoplankton["Route"] == 'PKME')|
        (phytoplankton["Route"] == 'MEAD')), :]

In [ ]:
#ANHO, BRSY, HOAN, MEAD, PKME

In [ ]:
#rounding coordinates
phyto_main_routes = phyto_main_routes.round(2)
phyto_main_routes.head()

In [ ]:
# get list of unique routes
column_values = phyto_main_routes[["Route"]].values.ravel()
unique_routes =  pd.unique(column_values)

print(unique_routes)

In [ ]:
phytoplankton.count

In [ ]:
phyto_main_routes.count


In [ ]:
for col in zooplankton.columns:
    print(col)

In [ ]:
#drop unneeded columns for zooplankton
zooplankton = zooplankton.drop(["id",
                                "FID",
                                "Time_24hr",
                                "family",
                                "Month",
                                "Day",
                                "genus",
                                "species",
                                "sex",
                                  "life_stage",
                                    "caab_code",
                                    "aphiaid",  
                                   "taxon_start_date",
                                   "zoop_comments",
                                   "acknowledgements",
                                   "first_occurrence",
                                "taxon_name",
                                   "parent_taxon_name",
                                   "training",
                                   "comments",
                                    "taxon_eco_group",
                                "geom"
                                  ],
                                  axis=1)

In [ ]:
zooplankton.head(20)

In [ ]:
zooplankton.count

In [ ]:
column_values = zooplankton[["Route"]].values.ravel()
unique_routes =  pd.unique(column_values)

print(unique_routes)

In [ ]:
#for loop to check which routes to drop

for route in unique_routes:
    print(route)
    route_filter = zooplankton.loc[zooplankton['Route'] == route]
    print(route_filter["Year"].value_counts())
    print("***************************************************")

In [ ]:
#ANHO, BRSY, HOAN, MEAD, PKME

In [ ]:
zoo_main_routes = zooplankton.loc[((zooplankton['Route'] == "ANHO") |
        (zooplankton["Route"] == 'BRSY')|
        (zooplankton["Route"] == 'HOAN')|
#        (zooplankton["Route"] == 'PKME')|
        (zooplankton["Route"] == 'MEAD')), :]

In [ ]:
zoo_main_routes.head()

In [ ]:
#rounding coordinates
zoo_main_routes = zoo_main_routes.round(2)
zoo_main_routes.head()

In [ ]:
# save to csv
zoo_main_routes.to_csv("clean_data/zooplankton_clean.csv")
phyto_main_routes.to_csv("clean_data/phytoplankton_clean.csv")

In [ ]:
zoo_main_routes.count

In [ ]:
for col in phytoplankton_color.columns:
    print(col)

In [ ]:
#drop unneeded columns for phytoplankton color index
phytoplankton_color = phytoplankton_color.drop(["FID",
                                                "start_port",
                                                "end_port",
                                                "trip_code",
                                                "colour_index",
                                                "measurement",
                                               "acknowledgements",
                                               "route_frequency",
                                               "route_start_date",
                                               "pci_comments",
                                                "geom",
                                                  ],
                                                  axis=1)

In [ ]:
phytoplankton_color

In [ ]:
# Create an engine for the plankton.sqlite database
engine = create_engine("sqlite:///planktontest.sqlite", echo=False)
conn = engine.connect()

In [ ]:
# add csv to data base function
# def read_sql(data):
#     data.to_sql(f"'{data}'", conn, if_exists = "append")


In [ ]:
# # add csv to data base
# read_sql(phytoplankton)
# read_sql(zooplankton)
# read_sql(phytoplankton_color)

In [ ]:
phyto_main_routes.to_sql("phytoplankton", conn,)
zoo_main_routes.to_sql("zooplankton", conn,)
phytoplankton_color.to_sql("phytoplankton_color_index", conn,)

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

In [ ]:
# Use `engine.execute` to select and display the first 10 rows from the station table
engine.execute('SELECT * FROM zooplankton LIMIT 5').fetchall()

In [ ]:
# Query phytoplankton Records in the the Database
data = pd.read_sql("SELECT * FROM phytoplankton", conn)

In [ ]:
data.head()